# Step 1: Compute the texture descriptions for the training images.

For each training image, calculate a vector of GLCM features.  Which GLCM features and the set of displacements you choose to you use are up to you (note that displacements for `skimage.feature.graycomatrix()` need to be specified by distances and angles in radians rather than change in x and y directions).  Experiment to obtain the best possible classification rate.  Use conservative choices to begin with until everything is working, then come back and experiemnt.  As described in the Topic 10 lecture notes, use `skimage.feature.graycomatrix()` and `skimage.feature.graycoprops()` to calculate GLCM features.  You'll probably want to use `normed=True` with `graycomatrix`.  Your GLCM features should be stored as a 120-row array by m-element array, (m will depend on how many different features and displacements you used and whether or not you combine values for different displacements or not, e.g., by taking their mean).  

_Hint: Pay close attention to the format of the return values of  `graycomatrix()` and `graycoprops()`._

Also, for each training image, calculate the rotationally invariant LBP features using `skiamge.feature.local_binary_pattern()`.  You can experiment with parameters `P` and `R` to get a good classification rate, but probably `P=8` and `R=1` are good enough.   For the `method` parameter, use `'uniform'` which gives you the LBP flavour we talked about in class.   Remember that `skiamge.feature.local_binary_pattern()` returns an "LBP Image", which is an image in which the pixel value is between 0 and 9, and corresponds to one of the ten possible pattern labels.  It's up to you to turn the "LBP Image" into a 10-bin histogram, which serves as the feature vector for that image (you can use `numpy.histogram` for this but again remember to specify `bins` and `range` parameters, and that it returns two things, and you only need the first one). 

Addionally, calculate the LBP variance feature again using `skimage.feature.local_binary_pattern()` but use `method='var'` instead.  This is the VAR feature we saw in class.  Use the same P and R as before.  Build a 16-bin histogram of the resulting 'LBP-VAR' image; use `range=(0,7000)` with `numpy.hisotgram()` (this is not quite "correct", but it's good enough).  Concatenate these with the rotationally invariant LBP features so that you have a 26-element feature vector for each training image.   These should be stored as a 120-row, 26-column array.

You can do this all in one loop which builds both feature arrays.



In [7]:
# Write your code here.

import skimage.feature as feat
import skimage.util as util
import glob
import skimage.io as io
import numpy as np
import os

trn_images = [os.path.basename(x) for x in glob.glob('brodatztraining/*.png')]
trn_images.sort()
trn_feats = []
lbp_trn = np.array([])
for i in range(0, len(trn_images)):
    B = util.img_as_ubyte(io.imread('brodatztraining/' + trn_images[i]))
    gclms = feat.greycomatrix(B, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], normed=True)
    #stats = []
    #stats.append(feat.greycoprops(gclms, prop='energy'))
    #stats.append(feat.greycoprops(gclms, prop='contrast'))
    #stats.append(feat.greycoprops(gclms, prop='correlation'))
    #stats.append(feat.greycoprops(gclms, prop='homogeneity'))
    #trn_feats.append(stats)
    trn_feats.append(feat.greycoprops(gclms, prop='energy'))
    #trn_feats.append(feat.greycoprops(gclms, prop='contrast'))
    lbp1 = feat.local_binary_pattern(B, 8, 1, method='uniform')
    lbp2 = feat.local_binary_pattern(B, 8, 1, method='var')
    lbp3 = np.concatenate((lbp1, lbp2))
    hist, other_stuff = np.histogram(lbp1, bins=26, range=(0, 7000))
    lbp_trn = np.concatenate((lbp_trn,hist))
    #lbp3 = np.concatenate(lbp1,lbp2)
    #lbp_trn.append(lbp3)
#print(trn_feats.shape)
trn_feats = np.transpose(np.asarray(trn_feats), (0,2,1))[:,:,0]
#print(trn_feats.shape)
lbp_trn = lbp_trn.reshape(-1, 1)

# Step 2: Compute Test Image Features

Compute the exact same features as you did in step 1 for each of the test images.  Store them in the same way (these arrays will just have more rows, specifically 320 rows, one for each testing sample). 

In [8]:
# Write your code here.  
tst_images = [os.path.basename(x) for x in glob.glob('brodatztesting/*.png')]
tst_images.sort()
tst_feats = []
lbp_tst = np.array([])
for i in range(0, len(tst_images)):
    B = util.img_as_ubyte(io.imread('brodatztesting/' + tst_images[i]))
    gclms = feat.greycomatrix(B, [1], [0, np.pi/4, np.pi/2, 3*np.pi/4], normed=True)
    #stats = []
    #stats.append(feat.greycoprops(gclms, prop='energy'))
    #stats.append(feat.greycoprops(gclms, prop='contrast'))
    #stats.append(feat.greycoprops(gclms, prop='correlation'))
    #stats.append(feat.greycoprops(gclms, prop='homogeneity'))
    #tst_feats.append(stats)
    tst_feats.append(feat.greycoprops(gclms, prop='energy'))
    lbp1 = feat.local_binary_pattern(B, 8, 1, method='uniform')
    lbp2 = feat.local_binary_pattern(B, 8, 1, method='var')
    lbp3 = np.vstack((lbp1, lbp2)).T
    hist, other_stuff = np.histogram(lbp1, bins=26, range=(0, 7000))
    lbp_tst = np.concatenate((lbp_tst, hist))
tst_feats = np.transpose(np.asarray(tst_feats), (0, 2, 1))[:, :, 0]
#tst_feats = np.asarray(tst_feats)

# Step 3: Generate Label Arrays for the Training and Testing Data

Use labels 1 for the first class, label 2 for the second class, etc.   This should be easy to do since the filenames are ordered in blocks of 15 or 40 images of each class for training and testing respectively.

In [9]:
# Write your code for step 3 here.  

trn_labels = np.zeros(len(trn_images))
clazz = 1
for i in range(0,len(trn_images)):
    if (i%15 == 0):
        clazz += 1
    trn_labels[i] = clazz

tst_labels = np.zeros(len(tst_images))
clazz = 1
for i in range(0,len(tst_images)):
    if (i%40 == 0):
        clazz += 1
    tst_labels[i] = clazz

# Step 4:  Train an KNN classifier.  

Train an KNN  classifier using your GLCM features.  Train another one using your LBP features.



In [10]:
import sklearn.neighbors as knn

# Write your code here. This should be quite short.

import sklearn.neighbors as neighbors
neighbs = 3
knn = neighbors.KNeighborsClassifier(n_neighbors=neighbs)
lnn = neighbors.KNeighborsClassifier(n_neighbors=neighbs)
#print("trn_feats: ", trn_feats.shape,"trn_labels: ", trn_labels.shape)
knn.fit(trn_feats, trn_labels)
lnn.fit(trn_feats, trn_labels)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

# Step 4:  Predict the classes of the test images

Predict the classes of the test images using both classifiers.

In [11]:
# Write your code here.  Again this should be quite short.
predict_labels = knn.predict(tst_feats)
lpredict_labels = lnn.predict(tst_feats)

# Step 6:  Display Results

Display results as in the final step of Question 1.  For each classifier display the image filenames that were incorrectly classified, the confisuion matrix, and the classification rate.  





In [13]:
# Write your code here.
confusion = np.zeros((clazz,clazz))
for i in range(0, len(tst_labels)):
    lt = int(tst_labels[i])
    l = int(predict_labels[i])
    confusion[lt-1, l-1] += 1

print("Confusion Matrix for graycomatrix\n", confusion)

print("Classification rate for graycomatrix: ", np.trace(confusion)/np.sum(confusion) * 100)

confusion = np.zeros((clazz,clazz))
for i in range(0, len(tst_labels)):
    lt = int(tst_labels[i])
    l = int(lpredict_labels[i])
    confusion[lt-1, l-1] += 1

print("Confusion Matrix for local binary pattern\n", confusion)

print("Classification rate for local binary pattern: ", np.trace(confusion)/np.sum(confusion) * 100)

Confusion Matrix for graycomatrix
 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  1. 31.  2.  0.  1.  4.]
 [ 0.  0.  0.  0. 32.  6.  0.  0.  2.]
 [ 0.  0.  3.  0. 31.  0.  0.  4.  2.]
 [ 0.  0.  0.  2. 33.  2.  0.  2.  1.]
 [ 0.  0.  3.  6.  0.  0.  0. 31.  0.]
 [ 0.  4.  0.  1. 32.  1.  0.  0.  2.]
 [ 0.  0.  0.  3. 37.  0.  0.  0.  0.]
 [ 0.  0. 15.  2.  0.  0.  0. 20.  3.]]
Classification rate for graycomatrix:  11.5625
Confusion Matrix for local binary pattern
 [[ 0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  1. 31.  2.  0.  1.  4.]
 [ 0.  0.  0.  0. 32.  6.  0.  0.  2.]
 [ 0.  0.  3.  0. 31.  0.  0.  4.  2.]
 [ 0.  0.  0.  2. 33.  2.  0.  2.  1.]
 [ 0.  0.  3.  6.  0.  0.  0. 31.  0.]
 [ 0.  4.  0.  1. 32.  1.  0.  0.  2.]
 [ 0.  0.  0.  3. 37.  0.  0.  0.  0.]
 [ 0.  0. 15.  2.  0.  0.  0. 20.  3.]]
Classification rate for local binary pattern:  11.5625


# Step 7: Reflections

Answer the following questions right here in this block:

- Discuss the performance difference of the two different texture features.  Hypothesize reasons for observed differenes.
	
	_Your answer:_

- For each of your two classifiers, discuss the misclassified images.  Were there any classes that were particularly difficult to distinguish?  Do the misclassified images (over all classes) have anything in common that would cause them to be misclassified?  If so what do they ahve in common, and why do you think it is confusing the classifier?

	_Your answer:_